Praktikum 9 -ImageReconstruction

In [14]:
import os
import numpy as np
from scipy.interpolate import interp1d
from scipy.ndimage import rotate
import glob
import cv2
from imutils import paths
import imutils
import random

ModuleNotFoundError: No module named 'imutils'

## Aufgabe 1

In [15]:
RAW_ROWS = 227
RAW_COLUMNS = 198
RAW_LEVELS = 129

In [16]:
current_directory_path: str = os.getcwd()

In [17]:
def load_raw_file(file_name: str, levels, rows, columns):
    raw = np.fromfile(file_name, dtype='int8')
    numpy_raw = np.reshape(raw, (levels, rows, columns))
    return numpy_raw

In [18]:
def extract_xy_middle_slice(raw):
    save_to_PGM(raw[int(RAW_LEVELS / 2), :, :], RAW_COLUMNS, RAW_ROWS, "xy_middle_slice")

In [19]:
def extract_xz_middle_slice(raw):
    save_to_PGM(raw[:, int(RAW_ROWS / 2), :], RAW_COLUMNS, RAW_LEVELS, "xz_middle_slice")

In [20]:
def extract_yz_middle_slice(raw):
    save_to_PGM(raw[:, :, int(RAW_COLUMNS / 2)], RAW_ROWS, RAW_LEVELS, "yz_middle_slice")

In [21]:
def mip(raw):
    save_to_PGM(np.max(raw, axis=0), RAW_COLUMNS, RAW_ROWS, "maximum_intensity")

In [22]:
def save_to_PGM(image_array, width, height, file_name, folder="created_images_Aufgabe_1"):
    pgm_header = "P5\n" + " " + str(width) + " " + str(height) + " " + str(255) + "\n"
    file = open(current_directory_path + "/" + folder + "/" + file_name + ".pgm", 'wb')
    pgm_header_byte = bytearray(pgm_header, 'utf-8')
    file.write(pgm_header_byte)
    file.write(bytearray(image_array))
    file.close()

In [23]:
def slices(raw):
    extract_xy_middle_slice(raw)
    extract_xz_middle_slice(raw)
    extract_yz_middle_slice(raw)


raw = load_raw_file(current_directory_path + "/images_for_reconstruction/whatisit_129x227x198_8bit.raw",
                    RAW_LEVELS, RAW_ROWS,
                    RAW_COLUMNS)
slices(raw)

mip(raw)

## Aufgabe 2

In [15]:
ARTEMISIA_RAW_ROWS = 138
ARTEMISIA_RAW_COLUMNS = 138
ARTEMISIA_RAW_LEVELS = 145

In [16]:
raw = load_raw_file(current_directory_path + "/images_for_reconstruction/Artemisia_pollen_145x138x138_8bit.raw",
                    ARTEMISIA_RAW_LEVELS, ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_COLUMNS)

In [24]:
def rotate_test(raw, degree):
    # todo delete
    rotation_y_axis = np.asarray([
        [np.cos(degree), 0, np.sin(degree), 0],
        [0, 1, 0, 0],
        [-np.sin(degree), 0, np.cos(degree), 0],
        [0, 0, 0, 1]
    ])

    rotated = rotate(raw, angle=5)
    return rotated

    inverse = np.linalg.inv(raw)
    trg_arr = np.zeros(raw.shape)

    #Iteration LEVELS
    for lev_index, lev in enumerate(raw):
        #Iteration ROWS
        for row_index, row in enumerate(lev):
            #Iteration COLUMNS
            for col_index, col in enumerate(row):
                # l = raw[lev_index, row_index, col_index]
                src_pos = lev_index, row_index, col_index, 1
                trg_pos = np.matmul(inverse, src_pos)
                trg_arr[lev_index, row_index, col_index] = interp1d(raw, trg_pos)
    return trg_arr

In [25]:
# output = rotate_test(raw, 100)
output = rotate(raw, angle=200, mode='nearest', axes=(1, 2), reshape=False, order=0)
print("OUTPUT: " + str(np.max(output)))
print("OUTPUT_SHAPE: " + str(output.shape))
save_to_PGM(output[int(ARTEMISIA_RAW_LEVELS / 2), :, :], ARTEMISIA_RAW_COLUMNS, ARTEMISIA_RAW_ROWS, "xy_pollen_test_3",
            "created_movie_frames_Aufgabe_2/")


OUTPUT: 127
OUTPUT_SHAPE: (129, 227, 198)


In [29]:
def generate_output_images(raw):
    outputs = []
    for degree in range(int(360 / 1)):
        output = rotate(raw, angle=degree * 1, mode='nearest', axes=(1, 2), reshape=False, order=0)
        movie_frame_name = "movie_frame_"
        degree_string = str(degree * 1)
        print("DEGREE: " + degree_string)
        if len(degree_string) == 1:
            movie_frame_name += movie_frame_name + "00" + degree_string
        elif len(degree_string) == 2:
            movie_frame_name += movie_frame_name + "0" + degree_string
        else:
            movie_frame_name += movie_frame_name + degree_string

        middle_slice = output[:, :, int(ARTEMISIA_RAW_COLUMNS / 2)]
        outputs.append(middle_slice)
        save_to_PGM(middle_slice, ARTEMISIA_RAW_COLUMNS, ARTEMISIA_RAW_ROWS, movie_frame_name,
                    "created_movie_frames_Aufgabe_2")

    return outputs

In [30]:
def create_video(folder_name=current_directory_path + "/created_movie_frames_Aufgabe_2"):
    movie_frames = []
    [movie_frames.append(movie_frame) for movie_frame in sorted(glob.glob(folder_name + "/movie_frame*.pgm"))]

    # out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15,
    #                       (ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_COLUMNS))
    out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15,
                          (ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_COLUMNS))
    for movie_frame in movie_frames:
        out.write(cv2.imread(movie_frame))
    out.release()

In [31]:
outputs = generate_output_images(raw)


DEGREE: 0
DEGREE: 1
DEGREE: 2
DEGREE: 3
DEGREE: 4
DEGREE: 5
DEGREE: 6
DEGREE: 7
DEGREE: 8
DEGREE: 9
DEGREE: 10
DEGREE: 11
DEGREE: 12
DEGREE: 13
DEGREE: 14
DEGREE: 15
DEGREE: 16
DEGREE: 17
DEGREE: 18
DEGREE: 19
DEGREE: 20
DEGREE: 21
DEGREE: 22
DEGREE: 23
DEGREE: 24
DEGREE: 25
DEGREE: 26
DEGREE: 27
DEGREE: 28
DEGREE: 29
DEGREE: 30
DEGREE: 31
DEGREE: 32
DEGREE: 33
DEGREE: 34
DEGREE: 35
DEGREE: 36
DEGREE: 37
DEGREE: 38
DEGREE: 39
DEGREE: 40
DEGREE: 41
DEGREE: 42
DEGREE: 43
DEGREE: 44
DEGREE: 45
DEGREE: 46
DEGREE: 47
DEGREE: 48
DEGREE: 49
DEGREE: 50
DEGREE: 51
DEGREE: 52
DEGREE: 53
DEGREE: 54
DEGREE: 55
DEGREE: 56
DEGREE: 57
DEGREE: 58
DEGREE: 59
DEGREE: 60
DEGREE: 61
DEGREE: 62
DEGREE: 63
DEGREE: 64
DEGREE: 65
DEGREE: 66
DEGREE: 67
DEGREE: 68
DEGREE: 69
DEGREE: 70
DEGREE: 71
DEGREE: 72
DEGREE: 73
DEGREE: 74
DEGREE: 75
DEGREE: 76
DEGREE: 77
DEGREE: 78
DEGREE: 79
DEGREE: 80
DEGREE: 81
DEGREE: 82
DEGREE: 83
DEGREE: 84
DEGREE: 85
DEGREE: 86
DEGREE: 87
DEGREE: 88
DEGREE: 89
DEGREE: 90
DEGREE: 9

In [33]:
create_video()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
imread_('/Users/john/Downloads/Praktikum_9/created_movie_frames_Aufgabe_2/movie_frame_movie_frame_000.pgm'): can't read data: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/bitstrm.cpp:110: error: (-2:Unspecified error) Unexpected end of input stream in function 'readBlock'

imread_('/Users/john/Downloads/Praktikum_9/created_movie_frames_Aufgabe_2/movie_frame_movie_frame_001.pgm'): can't read data: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/bitstrm.cpp:110: error: (-2:Unspecified error) Unexpected end of input stream in function 'readBlock'

imread_('/Users/john/Downloads/Praktikum_9/created_movie_frames_Aufgabe_2/movie_frame_movie_frame_002.pgm'): can't read data: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/mo

In [ ]:
# def transform_array(src_array):
#     trgPos = lev, row, col, 1
#     srcPos = myproduct(invMat, trgPos)
#     trgArr(lev, row, col) = interpolNN(src_array, srcPos)

## Aufgabe 3

In [22]:
def create_panorama(pathToImages):
    # initialize images
    # print("Image Path:" + pathToImages)
    imagePaths = sorted(list(paths.list_images(pathToImages)))
    images = []
    # load images
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        images.append(image)

    #initializing image stitcher
    print("stitching images...")
    stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
    (status, stitched) = stitcher.stitch(images)

    #check if stitching worked
    if status == 0:
        print("cropping...")
        stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,
                                      cv2.BORDER_CONSTANT, (0, 0, 0))
        # convert to gray-scale and threshold
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        # find largest contour to get outline of stitched image
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

        minRect = mask.copy()
        sub = mask.copy()
        while cv2.countNonZero(sub) > 0:
            #detect minimal mask for output image
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)
        cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)
        # get output from minimal mask and stitched image
        stitched = stitched[y:y + h, x:x + w]
        outputPath = current_directory_path + "/result" + str(random.randint(0, 50)) + ".jpg"
        print("Writing result to: " + outputPath)
        cv2.imwrite(outputPath, stitched)
        cv2.imshow("Stitched", stitched)
        cv2.waitKey(0)
    else:
        print("Error({})".format(status))

In [24]:
pathToCustomImagesDirectory = current_directory_path + "/panorama_images"
create_panorama(pathToCustomImagesDirectory)
pathToGivenImagesDirecotry = current_directory_path + "/images_for_blending"
create_panorama(pathToCustomImagesDirectory)

NameError: name 'paths' is not defined